In [61]:
from datetime import datetime
import requests, bs4
import re
import pandas as pd
import os
import numpy as np

Need transform functions to: 

- take in scraped scores by quarter and create individual dataframes per team like the old scraper
- record team and opponent box score stats
- calculate team and opponent four factor stats

In [72]:
url = 'https://www.basketball-reference.com/boxscores/201910230PHI.html'
res = requests.get(url, 'html_parser')
res.raise_for_status()
soup = bs4.BeautifulSoup(res.text, 'lxml')

In [73]:
q1_box = soup.find('table',{'id':'box-BOS-game-basic'})

In [74]:
q1_df = pd.read_html(str(q1_box),flavor='bs4')[0]

In [75]:
players = q1_df['Unnamed: 0_level_0'].drop([5],axis=0).reset_index(drop=True)

In [76]:
players = players[:-1]

In [77]:
players.rename({'Starters':'players'}, axis=1)

,players
0,Jayson Tatum
1,Gordon Hayward
2,Kemba Walker
3,Enes Kanter
4,Jaylen Brown
5,Marcus Smart
6,Carsen Edwards
7,Robert Williams
8,Daniel Theis
9,Semi Ojeleye


In [78]:
team_q1 = q1_df['Basic Box Score Stats'].drop([5], axis =0).reset_index(drop=True)

In [80]:
team_q1 = team_q1[-1:]

In [81]:
team_q1

,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
13,240,33,90,.367,7,26,.269,20,34,.588,10,31,41,18,4,2,10,29,93,NaN


In [50]:
df = pd.read_csv('2020_line_scores.csv',index_col=0)

In [51]:
df.head()

,date,away_team,away_q1,away_q2,away_q3,away_q4,away_OT,away_final,home_team,home_q1,home_q2,home_q3,home_q4,home_OT,home_final
0,2019-10-22,NOP,30.0,31.0,25.0,31.0,5.0,122.0,TOR,27.0,29.0,32.0,29.0,13.0,130.0
1,2019-10-22,LAL,25.0,29.0,31.0,17.0,0.0,102.0,LAC,22.0,40.0,23.0,27.0,0.0,112.0
2,2019-10-23,DET,27.0,27.0,29.0,36.0,0.0,119.0,IND,24.0,31.0,31.0,24.0,0.0,110.0
3,2019-10-23,SAC,29.0,30.0,17.0,19.0,0.0,95.0,PHO,25.0,29.0,32.0,38.0,0.0,124.0
4,2019-10-23,CLE,24.0,17.0,24.0,20.0,0.0,85.0,ORL,28.0,27.0,16.0,23.0,0.0,94.0


In [88]:
bos = df[np.logical_or(df['home_team']=='BOS', df['away_team']=='BOS')].copy()

In [89]:
bos['Team']='BOS'

In [90]:
bos.head()

,date,away_team,away_q1,away_q2,away_q3,away_q4,away_OT,away_final,home_team,home_q1,home_q2,home_q3,home_q4,home_OT,home_final,Team
6,2019-10-23,BOS,25.0,23.0,20.0,25.0,0.0,93.0,PHI,20.0,29.0,28.0,30.0,0.0,107.0,BOS
16,2019-10-25,TOR,21.0,28.0,33.0,24.0,0.0,106.0,BOS,22.0,28.0,26.0,36.0,0.0,112.0,BOS
31,2019-10-26,BOS,22.0,24.0,36.0,36.0,0.0,118.0,NYK,24.0,27.0,25.0,19.0,0.0,95.0,BOS
64,2019-10-30,MIL,34.0,24.0,18.0,29.0,0.0,105.0,BOS,19.0,23.0,38.0,36.0,0.0,116.0,BOS
75,2019-11-01,NYK,23.0,27.0,24.0,28.0,0.0,102.0,BOS,22.0,27.0,26.0,29.0,0.0,104.0,BOS


In [98]:
bos['opponent'] = np.nan
bos['home_game'] = np.nan
bos['team_q1'] = np.nan
bos['team_q2'] = np.nan
bos['team_q3'] = np.nan
bos['team_q4'] = np.nan
bos['team_OT'] = np.nan
bos['team_final']=np.nan


In [99]:
for i in range(len(bos['opponent'])):
    
    if bos['away_team'].iloc[i] == 'BOS':
        bos['home_game'].iloc[i] = 0
        bos['opponent'].iloc[i] = bos['home_team'].iloc[i]
    
    else:
        bos['home_game'].iloc[i] = 1
        bos['opponent'].iloc[i] = bos['away_team'].iloc[i]

In [100]:
bos.head()

,date,away_team,away_q1,away_q2,away_q3,away_q4,away_OT,away_final,home_team,home_q1,home_q2,home_q3,home_q4,home_OT,home_final,Team,opponent,home_game
6,2019-10-23,BOS,25.0,23.0,20.0,25.0,0.0,93.0,PHI,20.0,29.0,28.0,30.0,0.0,107.0,BOS,PHI,0.0
16,2019-10-25,TOR,21.0,28.0,33.0,24.0,0.0,106.0,BOS,22.0,28.0,26.0,36.0,0.0,112.0,BOS,TOR,1.0
31,2019-10-26,BOS,22.0,24.0,36.0,36.0,0.0,118.0,NYK,24.0,27.0,25.0,19.0,0.0,95.0,BOS,NYK,0.0
64,2019-10-30,MIL,34.0,24.0,18.0,29.0,0.0,105.0,BOS,19.0,23.0,38.0,36.0,0.0,116.0,BOS,MIL,1.0
75,2019-11-01,NYK,23.0,27.0,24.0,28.0,0.0,102.0,BOS,22.0,27.0,26.0,29.0,0.0,104.0,BOS,NYK,1.0
